# 2b_NaiveRAG_k_experiment.ipynb

## Notebook Overview: Naive RAG `k` Experiments

This notebook explores how different `k` values (number of retrieved contexts) affect the performance of our RAG pipeline.
- Runs RAG inference with multiple `k` settings
- Measures accuracy for multiple choice questions
- Helps determine the optimal retrieval size for our final pipeline



In [ ]:
!apt install gh -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  gh
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 6,242 kB of archives.
After this operation, 33.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 gh amd64 2.4.0+dfsg1-2 [6,242 kB]
Fetched 6,242 kB in 1s (8,444 kB/s)
Selecting previously unselected package gh.
(Reading database ... 126109 files and directories currently installed.)
Preparing to unpack .../gh_2.4.0+dfsg1-2_amd64.deb ...
Unpacking gh (2.4.0+dfsg1-2) ...
Setting up gh (2.4.0+dfsg1-2) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
!gh auth login --hostname github.com --web



! First copy your one-time code: 842C-0FC4
- Press Enter to open github.com in your browser... 
/usr/bin/xdg-open: 882: www-browser: not found
/usr/bin/xdg-open: 882: links2: not found
/usr/bin/xdg-open: 882: elinks: not found
/usr/bin/xdg-open: 882: links: not found
/usr/bin/xdg-open: 882: lynx: not found
/usr/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening 'https://github.com/login/device'
! Failed opening a web browser at https://github.com/login/device
  exit status 3
  Please try entering the URL in your browser manually
✓ Authentication complete. Press Enter to continue...

✓ Logged in as LorenaRaichle


In [1]:

!git clone https://github.com/LorenaRaichle/ClinIQ.git


Cloning into 'ClinIQ'...
remote: Enumerating objects: 36079, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 36079 (delta 61), reused 69 (delta 32), pack-reused 35980 (from 1)
Receiving objects: 100% (36079/36079), 172.42 MiB | 29.07 MiB/s, done.
Resolving deltas: 100% (12912/12912), done.
Updating files: 100% (54/54), done.


In [2]:
!pip install pinecone sentence-transformers transformers datasets
! pip install spacy
! pip install langchain_community
! pip install langchain_pinecone
!pip install -U langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.3/516.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.1/239.1 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitli

In [3]:

import os
import sys

%cd /content/ClinIQ

sys.path.append(os.getcwd())

import spacy
nlp = spacy.load("models/en_ner_bc5cdr_md-0.5.4")
print(nlp.pipe_names)


/content/ClinIQ


/usr/local/lib/python3.11/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_ner_bc5cdr_md' (0.5.4) was trained with spaCy v3.7.4 and may not be 100% compatible with the current version (3.8.6). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.11/dist-packages/spacy/language.py:2232: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


['tok2vec', 'tagger', 'attribute_ruler', 'lemmatizer', 'parser', 'ner']


In [4]:
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec
from config import INDEX_NAME, DIMENSION
import sys
from pathlib import Path

#sys.path.append(str(Path(".").resolve()))
from utils.RAG_metadata import extract_keywords_and_entities, extract_age_gender
from utils.RAG_pinecone import init_index, RAGIndexer
from pathlib import Path
import os

pinecone_key = userdata.get("PINECONE")
pc = Pinecone(api_key=pinecone_key, environment="us-west1-gcp")

index = init_index(pc)

/usr/local/lib/python3.11/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_ner_bc5cdr_md' (0.5.4) was trained with spaCy v3.7.4 and may not be 100% compatible with the current version (3.8.6). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


ℹ Index 'rag-trainset-index' already exists.


In [5]:
indexer = RAGIndexer(index)
indexer.describe_index()


/usr/local/lib/python3.11/dist-packages/sentence_transformers/SentenceTransformer.py:196: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v4 of SentenceTransformers.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 411857}},
 'total_vector_count': 411857,
 'vector_type': 'dense'}


{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 411857}},
 'total_vector_count': 411857,
 'vector_type': 'dense'}

In [6]:
%cd /content/ClinIQ
from utils.RAG_preprocessing import DataPaths, DataStats
data = DataPaths.load("test_processed")
stats = DataStats.get_stats("test_processed", show_sources=False)

/content/ClinIQ

Stats for 'test_processed':
 - true_false: 42254 examples
 - multiple_choice: 43466 examples
 - short_answer: 13516 examples
 - multi_hop: 3941 examples
Total entries: 103177


In [12]:
from utils.RAG_naive_pipeline import RAGPipeline
from utils.prompt_utils import generate_model_input
import pprint
from datasets import Dataset, DatasetDict

import json
from utils.RAG_preprocessing import DataPaths

with open(DataPaths.paths["test_processed"], "r") as f:
    test_data_raw = json.load(f)

testset = DatasetDict({
    split: Dataset.from_list(split_data)
    for split, split_data in test_data_raw.items()
})
mc_testset = testset["multiple_choice"].to_list()



with open(DataPaths.paths["train_processed"], "r") as f:
    full_data = json.load(f)

full_data = DatasetDict({
    split: Dataset.from_list(split_data)
    for split, split_data in full_data.items()
})

json_file_path = '/content/drive/MyDrive/NLP/04_RAG/TopicModeling/PubMed_raw/pubmed_sample_prepro.jsonl'
pubmed_data = {}
with open(json_file_path, 'r') as file:
    for line in file:
        doc = json.loads(line)
        doc_id = doc.get("id")
        if doc_id:
            pubmed_data[doc_id] = doc


pprint.pprint(mc_testset[0])



{'correct_answer': 'B',
 'id': 'mc_0',
 'options': {'A': 'Caput',
             'B': 'Chignon',
             'C': 'Phlegmon',
             'D': 'None of the above.',
             'E': None},
 'question': 'Formation of one of the following is essential for success of '
             'this instrument?',
 'source': 'MC3-openlifescienceai/medmcqa',
 'type': 'multiple_choice'}


In [12]:
%cd /content/ClinIQ
!git pull origin main


/content/ClinIQ
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 5 (delta 4), reused 5 (delta 4), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 681 bytes | 681.00 KiB/s, done.
From https://github.com/LorenaRaichle/ClinIQ
 * branch              main       -> FETCH_HEAD
   c65ec1a3..3255a437  main       -> origin/main
Updating c65ec1a3..3255a437
Fast-forward
 utils/RAG_naive_pipeline.py | 64 +++++++++++++++++++++++----------------------
 utils/RAG_preprocessing.py  |  3 +--
 2 files changed, 34 insertions(+), 33 deletions(-)


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from langchain_community.llms import HuggingFacePipeline
import torch

def load_deepseek_pipeline():
    model_id = "deepseek-ai/deepseek-coder-7b-instruct-v1.5"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype=torch.float16
    )
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
    return HuggingFacePipeline(pipeline=pipe)

shared_model_pipeline = load_deepseek_pipeline()

tokenizer_config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/621 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-10-6fedf938b77f>:14: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  return HuggingFacePipeline(pipeline=pipe)


In [14]:
import json
import copy
from tqdm import tqdm
import pprint
import torch
import gc

from datasets import Dataset, DatasetDict
from utils.prompt_utils import generate_model_input
from utils.RAG_preprocessing import DataPaths
from utils.RAG_answer import extract_multiple_choice_letters
from langchain_huggingface import HuggingFaceEmbeddings
from config import EMBEDDING_MODEL_NAME
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from langchain_community.llms import HuggingFacePipeline


# Shared embedding pipe for all runs
shared_embedding_pipe = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)

k_values = [1, 3, 5, 8, 12]

for k in k_values:
    print(f"\n=== Running for k={k} ===")
    torch.cuda.empty_cache()
    gc.collect()


    eval_examples = mc_testset[:300] # eval for num questions

    pipeline = RAGPipeline(
    test_data=mc_testset,
    full_data=full_data,
    pubmed_data=pubmed_data,
    k=k,
    pc=pc,
    embedding_pipe=shared_embedding_pipe,
    model_pipeline=shared_model_pipeline
)

    predictions = []
    for example in tqdm(eval_examples, desc=f"Generating answers for k={k}"):
        prepared_example = generate_model_input(copy.deepcopy(example))
        response = pipeline.run(prepared_example)
        predictions.append(response)

    # Extract multiple-choice letter answers and attach to predictions
    predicted_letters = extract_multiple_choice_letters(predictions)
    for i, pred in enumerate(predictions):
        pred['predicted_letter'] = predicted_letters[i]

    output_file_path = DataPaths.get_k_experiment_output_path(k)
    with open(output_file_path, 'w') as output_file:
        json.dump(predictions, output_file, indent=4)

    print(f"Saved results for k={k} to {output_file_path}")



=== Running for k=1 ===
Using shared model pipeline.


Generating answers for k=1: 100%|██████████| 300/300 [37:30<00:00,  7.50s/it]


Saved results for k=1 to /content/ClinIQ/content/k_experiments/k_1/MC_1000_generated_answers_k1.json

=== Running for k=3 ===
Using shared model pipeline.


Generating answers for k=3: 100%|██████████| 300/300 [36:45<00:00,  7.35s/it]


Saved results for k=3 to /content/ClinIQ/content/k_experiments/k_3/MC_1000_generated_answers_k3.json

=== Running for k=5 ===
Using shared model pipeline.


...

Saved results for k=5 to /content/ClinIQ/content/k_experiments/k_5/MC_1000_generated_answers_k5.json

=== Running for k=8 ===
Using shared model pipeline.


Generating answers for k=8: 100%|██████████| 300/300 [37:59<00:00,  7.60s/it]


Saved results for k=8 to /content/ClinIQ/content/k_experiments/k_8/MC_1000_generated_answers_k8.json

=== Running for k=12 ===
Using shared model pipeline.


Generating answers for k=12: 100%|██████████| 300/300 [36:05<00:00,  7.22s/it]

Saved results for k=12 to /content/ClinIQ/content/k_experiments/k_12/MC_1000_generated_answers_k12.json
